In [7]:
import numpy as np
import pandas as pd 
import csv 
from sklearn.preprocessing import OneHotEncoder 
from sklearn.model_selection import train_test_split
import warnings
warnings.simplefilter("ignore")

from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, classification_report, confusion_matrix

from sklearn import metrics

In [8]:
#  loading the data set 
file = pd.read_csv('BSOM_DataSet_for_HW3.csv')


# to check whether there are any empty values or not 
df=file[['all_mcqs_avg_n20', 'all_NBME_avg_n4', 'CBSE_01',  'CBSE_02', 'LEVEL']]


df['LEVEL'].fillna(value='A', inplace=True) 

df[['CBSE_02']].fillna(method ='bfill')
#.fillna(52, inplace=True)
df.fillna(df['CBSE_02'].mean(), inplace=True)

df.isnull().any()


all_mcqs_avg_n20    False
all_NBME_avg_n4     False
CBSE_01             False
CBSE_02             False
LEVEL               False
dtype: bool

In [9]:
# Mapping LEVEL column to 0 ,1,2,3 

LEVEL = {'A': 0,'B': 1,'C':2,'D':3}

df.LEVEL = [LEVEL[item] for item in df.LEVEL] 
print(df) 

     all_mcqs_avg_n20  all_NBME_avg_n4  CBSE_01  CBSE_02  LEVEL
0               0.736           0.7700       42     68.0      1
1               0.740           0.8000       44     67.0      2
2               0.807           0.8125       41     78.0      0
3               0.886           0.9250       68     91.0      0
4               0.839           0.8550       57     74.0      1
..                ...              ...      ...      ...    ...
110             0.637           0.6825       45     52.0      2
111             0.751           0.8150       53     60.0      2
112             0.811           0.8325       50     77.0      1
113             0.729           0.7275       43     68.0      2
114             0.839           0.8625       54     73.0      1

[115 rows x 5 columns]


In [10]:
# Transforming y to binary number using one-hot- encoding
from sklearn.compose import ColumnTransformer

#onhtencder = OneHotEncoder(categorical_features = [0]) 

onhtencder= ColumnTransformer([("LEVEL", OneHotEncoder(), [0])], remainder = 'passthrough')


data = onhtencder.fit_transform(df[['LEVEL']]).toarray() 


In [11]:

X= df.iloc[:,:]

#Normalizing the data 

X_norm = (X.iloc[:,:-1] - X.iloc[:,:-1].mean()) / (X.iloc[:,:-1].max() - X.iloc[:,:-1].min())



X_modified = pd.concat([X_norm, X.iloc[:,-1]],axis=1)

target=df.iloc[:,-1]

#Train test split 
XTrain, XTest, yTrain, yTest = train_test_split(X_modified, data, test_size=0.20,random_state=9)


Train_ip=np.array(XTrain.iloc[:,:-1])
Train_otpts=np.array(yTrain)

In [12]:

np.random.seed(22)

wt1 = np.random.random((5,5))    
wt2 = np.random.random((4,6))    
ttl_weits = np.hstack((wt1.ravel(order='F'), wt2.ravel(order='F')))    #unrolling the  params

# layers configuration 

ip_lyr = 4
hd_lyr = 5
no_labls = 4

#print('wt1',wt1)
#print('wt2',wt2)
#print('ttl_weits',ttl_weits)
#print(ttl_weits.shape)

In [13]:
def Weights(ttl_weits,ip,hidn,otpt):
    w1 = np.reshape(ttl_weits[:hidn*(ip+1)], (hidn, ip+1), 'F')
    w2 = np.reshape(ttl_weits[hidn*(ip+1):], (otpt, hidn+1), 'F')
    return w1,w2

    

In [14]:
def SIG(z):
    return 1 / (1 + np.exp(-z))


def SIGGrad(z):
    return np.multiply(SIG(z), 1-SIG(z))



In [15]:
def Cost(h,y):
    cost = np.sum(np.multiply(y, np.log(h))+np.multiply(1-y, np.log(1-h)))
    return cost

In [16]:
# Forward_propagation 

def Feedfrwd(ttl_weits, ip_lyr, hd_lyr, no_labls, Train_ip, Train_otpts):
    
    wt1,wt2= Weights(ttl_weits, ip_lyr, hd_lyr, no_labls)

   
    l = len(Train_ip)
   
    ones = np.ones((l,1))
    #print('ones',ones)
    a_1 = np.hstack((ones, Train_ip))
    #print('a_1',a_1)
    a_2 = SIG(a_1 @ wt1.T)
    #print('a_2',a_2)
    a_2 = np.hstack((ones, a_2))
    #print('a_2',a_2)
    h = SIG(a_2 @ wt2.T)
    #print('h',h)
    
    c=Cost(h,Train_otpts)

   
    
    return c / (-l)    
    

In [17]:
cf_er = Feedfrwd(ttl_weits, ip_lyr, hd_lyr, no_labls, Train_ip,Train_otpts)


In [18]:


def NN(ttl_weits, ip_lyr, hd_lyr, no_labls, Train_ip,Train_otpts):
    
    
    initt_wt1,initt_wt2= Weights(ttl_weits, ip_lyr, hd_lyr, no_labls)
    
   
    lenth = len(Train_otpts)

    ones = np.ones((lenth,1))
    # Farward Prop
    
    a_1 = np.hstack((ones, Train_ip))
    #print('a_1',a_1)
    #print('a_1 shape',a_1.shape)
    z_2 = a_1 @ initt_wt1.T
    #print('z_2',z_2)
    #print('z_2 sha',z_2.shape)
    a_2 = np.hstack((ones, SIG(z_2)))
    #print('a_2 ',a_2)
    #print('a_2 ',a_2.shape)
    z_3 = a_2 @ initt_wt2.T
    #print('z_3',z_3)
    #print('z_3 ',z_3.shape)
    a_3 = SIG(z_3)
    #print('a_3',a_3)
    #print('a_3',a_3.shape)
    
    
    #Backword_propagation

    d_3 = a_3 - Train_otpts
    #print('d_3',d_3)
    #print('d_3',d_3.shape)
    z_2 = np.hstack((ones, z_2))
    #print('z_2',z_2)
    #print('z_2',z_2.shape)
    d_2 = np.multiply(initt_wt2.T @ d_3.T, SIGGrad(z_2).T)
    #print('d_2',d_2)
    #print('d_2',d_2.shape)
    
    delta1 = np.zeros(initt_wt1.shape)
    #print('delta1',delta1)
    #print('delta1 shape',delta1.shape)
    delta2 = np.zeros(initt_wt2.shape)
    #print('delta2',delta2)
    #print('delta2 shape',delta2.shape)
    
    delta1 = delta1 + d_2[1:,:] @ a_1
    #print('delta1',delta1)
    #print('delta1',delta1.shape)
    delta2 = delta2 + d_3.T @ a_2
    #print('delta2',delta2)
    #print('delta2 ',delta2.shape)
        
        
    delta1 /= lenth
    delta2 /= lenth
    
    Deltas=np.hstack((delta1.ravel(order='F'), delta2.ravel(order='F')))
   
    return Deltas

In [19]:
bckprop_wts = NN(ttl_weits, ip_lyr, hd_lyr, no_labls,Train_ip,Train_otpts)

#bckprop_wts

In [20]:
def predict(wt1, wt2, X, y):
    l = len(y)
    ones = np.ones((l,1))
    a_1 = np.hstack((ones, X))
    a_2 = SIG(a_1 @ wt1.T)
    a_2 = np.hstack((ones, a_2))
    h = SIG(a_2 @ wt2.T)
    return np.argmax(h, axis = 1) 

In [21]:

def checkGradient(ttl_weits,bckprop_wts,iplyr, hdlyr, no_labls,Train_ip,Train_otpts):
    eps = 0.22
   
    
    for i in range(5):
        iterattion = int(np.random.rand()*len(ttl_weits))
        
        adding_eps = np.zeros((len(ttl_weits),1))
        
        adding_eps[iterattion] = eps
       

        ThetaPlus = Feedfrwd(ttl_weits + adding_eps.flatten(),iplyr, hdlyr, no_labls,Train_ip,Train_otpts)
        
        
        
        ThetaMinus  = Feedfrwd(ttl_weits - adding_eps.flatten(),iplyr, hdlyr, no_labls,Train_ip,Train_otpts)
        
        
        num_app = (ThetaPlus - ThetaMinus) / float(2*eps)
        print('num_app',num_app)
       
        print('bckprop_wts[iterattion]',bckprop_wts[iterattion])
        
        



In [22]:
result=checkGradient(ttl_weits,bckprop_wts,ip_lyr, hd_lyr, no_labls,Train_ip,Train_otpts)

result# To check whether the numerical approximation and our back prop should be similar for atleast 
#first 3 digists such that our back propagation is appropriate 

num_app -0.012884943617861501
bckprop_wts[iterattion] -0.012885842830204258
num_app -0.010816391146476175
bckprop_wts[iterattion] -0.010826531600240177
num_app 0.25373229674779246
bckprop_wts[iterattion] 0.2538121914930991
num_app 0.30795798493166765
bckprop_wts[iterattion] 0.3080374231787549
num_app 0.003043225082133283
bckprop_wts[iterattion] 0.003043060435647263


In [23]:
import scipy.optimize as opt
Final_wts = opt.fmin_cg(maxiter = 100, f = Feedfrwd, x0 = ttl_weits, fprime = NN,args = (ip_lyr, hd_lyr, no_labls,Train_ip,Train_otpts))


Final_wts1,Final_wts2= Weights(Final_wts, ip_lyr, hd_lyr, no_labls)

         Current function value: 0.993975
         Iterations: 100
         Function evaluations: 243
         Gradient evaluations: 243


## Predicting y values for test_data

In [24]:
pred = predict(Final_wts1, Final_wts2, XTest.iloc[:,:-1], yTest)
pred


array([2, 0, 0, 1, 1, 1, 2, 1, 1, 2, 1, 1, 0, 1, 1, 0, 1, 1, 0, 1, 1, 1,
       0], dtype=int64)

In [25]:
y_actual=XTest.iloc[:,-1]
y_actual=np.array(y_actual)

print('The confusion matrix is :')
print(  confusion_matrix(y_actual,pred))

#print(accuracy_score(y_actual,pred))

#from sklearn.metrics import accuracy_score
#accuracy_score(y_actual,pred)
#print(type(y_test))

#print(accuracy_score(y_actual,pred))


The confusion matrix is :
[[3 1 1 0]
 [3 8 0 0]
 [0 4 2 0]
 [0 1 0 0]]


In [26]:


print('f1_score',f1_score(y_actual,pred, average="weighted"))
print('precision_score',precision_score(y_actual,pred, average="weighted"))
print('recall_score',recall_score(y_actual,pred, average="weighted"))  

fpr, tpr, thresholds = metrics.roc_curve(y_actual,pred, pos_label=2)
print('AUC score :',metrics.auc(fpr, tpr))


f1_score 0.5406060606060606
precision_score 0.5559006211180124
recall_score 0.5652173913043478
AUC score : 0.7549019607843137
